In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%aimport sys
sys.path.insert(0,'..')

In [3]:
%env TEXTMORPH_DATA=/Users/guillaume/Desktop/backup/neural-editor-grll-data

env: TEXTMORPH_DATA=/Users/guillaume/Desktop/backup/neural-editor-grll-data


In [12]:
from os.path import dirname, realpath, join
import cPickle as pickle
import logging
import random
from collections import OrderedDict
import torch
import torch.optim as optim
from textmorph import data
from textmorph.edit_model.training_run import EditTrainingRuns,EditTrainingRun,TrainState, RandomState
from gtd.utils import Config, random_seed, chunks
from gtd.chrono import verboserate

In [5]:
class MyEditTrainingRuns(EditTrainingRuns):
    def __init__(self):
        data_dir = data.workspace.edit_runs
        src_dir = dirname(dirname(dirname(realpath('__file__'))))
        super(EditTrainingRuns, self).__init__(data_dir, src_dir, MyEditTrainingRun, check_commit=False)
        
    def __getitem__(self, i):
        """Reload an existing TrainingRun."""
        save_dir = self._int_dirs[i]
        config = Config.from_file(self._config_path(save_dir))
        run = self._run_factory(config, save_dir)
        if self._check_commit:
            run.match_commit(self._src_dir)

        logging.info('Reloaded TrainingRun #{}'.format(i))
        return run

class MyEditTrainingRun(EditTrainingRun):
    def __init__(self, config, save_dir):
        super(MyEditTrainingRun, self).__init__(config, save_dir)
    
    def _initialize_train_state(cls, config):
        """Set up all the state necessary to begin training."""
        with random_seed(config.seed):
            editor = cls._build_editor(config.editor)
            optimizer = optim.Adam(editor.parameters(), lr=config.optim.learning_rate)
            train_steps = 0
            max_grad_norm = 0
            random_state = RandomState()

        return MyTrainState(editor=editor, optimizer=optimizer, train_steps=train_steps,
                          random_state=random_state, max_grad_norm=max_grad_norm)
    
    def _reload_train_state(cls, checkpoints_dir, train_steps, config):
        ckpt_path = join(checkpoints_dir, '{}.checkpoint'.format(train_steps))
        init_state = cls._initialize_train_state(config)
        editor, optimizer = init_state.editor, init_state.optimizer
        return MyTrainState.load(init_state, ckpt_path, editor, optimizer)
    

class MyTrainState(TrainState):
    def __init__(self, editor, optimizer, train_steps, random_state, max_grad_norm):
        super(MyTrainState, self).__init__( editor, optimizer, train_steps, random_state, max_grad_norm)
    
    def load(cls, path, editor, optimizer):
        with open(join(path, 'metadata.p'), 'r') as f:
            d = pickle.load(f)
        
        # load model
        optimizer.load_state_dict(torch.load(join(path, 'optimizer'),  map_location='cpu'))
        
        new_state_dict = OrderedDict({})
        for key, value in torch.load(join(path, 'editor'), map_location='cpu').items():
            new_state_dict[key] = value.squeeze()
        
        new_state_dict["train_decoder.decoder_cell.source_attention.v_transform"] = new_state_dict["train_decoder.decoder_cell.source_attention.v_transform"].unsqueeze(dim=1)
        new_state_dict["train_decoder.decoder_cell.insert_attention.v_transform"] = new_state_dict["train_decoder.decoder_cell.insert_attention.v_transform"].unsqueeze(dim=1)
        new_state_dict["train_decoder.decoder_cell.delete_attention.v_transform"] = new_state_dict["train_decoder.decoder_cell.delete_attention.v_transform"].unsqueeze(dim=1)

        editor.load_state_dict(new_state_dict)
        train_state = MyTrainState(editor=editor, optimizer=optimizer, **d)
        return train_state

In [17]:
def similar_size_batches(examples, batch_size, size=lambda x: len(x.target_words)):
    """Create similar-sized batches of EditExamples.

    By default, elements with similar len('source_words') are batched together.
    See editor.py / EditExample.

    Args:
        examples (list[EditExample])
        batch_size (int)
        size (Callable[[EditExample], int])

    Returns:
        list[list[EditExample]]
    """
    assert batch_size >= 1
    sorted_examples = sorted(examples, key=size)
    batches = list(chunks(sorted_examples, batch_size))
    random.seed(1)
    random.shuffle(batches)
    # report savings
    suboptimal_batches = list(chunks(examples, batch_size))

    total_cost = lambda batches: batch_size * sum(max(size(b) for b in batch) for batch in batches)
    naive_cost = total_cost(suboptimal_batches)
    improved_cost = total_cost(batches)
    optimal_cost = sum(size(ex) for ex in examples)

    print 'Optimized batches: reduced cost from {naive} (naive) to {improved} ({reduction}% reduction).\n' \
          'Optimal (batch_size=1) would be {optimal}.'.format(naive=naive_cost, improved=improved_cost,
                                                              reduction=float(naive_cost - improved_cost) / naive_cost,
                                                              optimal=optimal_cost)

    return batches

In [7]:
experiments = MyEditTrainingRuns()
exp_id = [6]
exp = experiments[int(exp_id[0])]

Reloaded checkpoint #100000


SEJveChjaGlsZHJlbj0oSW50UHJvZ3Jlc3ModmFsdWU9MCwgZGVzY3JpcHRpb249dSdMb2FkaW5nIGVtYmVkZGluZ3MgZnJvbSAvVXNlcnMvZ3VpbGxhdW1lL0Rlc2t0b3AvYmFja3VwL25ldXLigKY=



No GPUs detected. Sticking with CPUs.
[localhost] local: wc -l /Users/guillaume/Desktop/backup/neural-editor-grll-data/de-amazon-review/train.tsv



[localhost] local: wc -l /Users/guillaume/Desktop/backup/neural-editor-grll-data/de-amazon-review/valid.tsv



[localhost] local: wc -l /Users/guillaume/Desktop/backup/neural-editor-grll-data/de-amazon-review/test.tsv


In [8]:
exp._examples.valid # found too

[SOURCE: meine erwartungen an das das empfehlungsmarketing wurden nicht enttäuscht .
 INSERT EXACT: [u'buch', u'erf\xfcllt']
 DELETE EXACT: [u'empfehlungsmarketing', u'entt\xe4uscht']
 TARGET: meine erwartungen an das buch wurden nicht erfüllt .,
 SOURCE: ich kann mich meinen vorredner nur anschließen .
 INSERT EXACT: [u'da']
 DELETE EXACT: [u'meinen', u'vorredner']
 TARGET: ich kann mich da nur anschließen .,
 SOURCE: was bitte hat das mit der handlung zu tun ?
 INSERT EXACT: [u'internationalen', u'kapitalistischen', u'neuorganisation', u'produktion']
 DELETE EXACT: [u'bitte', u'handlung']
 TARGET: was hat das mit der neuorganisation der internationalen kapitalistischen produktion zu tun ?,
 SOURCE: schlechtester film , den ich je gesehen habe
 INSERT EXACT: [u'<per>', u'beste', u'der']
 DELETE EXACT: [u',', u'schlechtester']
 TARGET: der beste film den ich je gesehen habe <per>,
 SOURCE: ich kann dieses buch nur weiterempfehlen !
 INSERT EXACT: [u'bekommen', u'es', u'geschenkt', u'ha

In [18]:
batch_size = 2
batches = similar_size_batches(exp._examples.valid, batch_size, size=lambda x: len(x.source_words))

Optimized batches: reduced cost from 2423074 (naive) to 2129408 (0.121195638268% reduction).
Optimal (batch_size=1) would be 2129359.


In [19]:
for batch in verboserate(batches, desc='Streaming Source Sentences'):
    loss = exp.editor.loss(batch, draw_samples=False)
    # Source Encode
    # source_words, insert_words, insert_exact_words, delete_words, delete_exact_words, target_words, edit_embed = exp.editor._batch_editor_examples(batch)
    # encoder_input = exp.editor.encoder.preprocess(source_words, insert_words, insert_exact_words, delete_words, delete_exact_words, edit_embed)
    # encoder_output = exp.editor.encoder(encoder_input)
    break

torch.Size([2, 3])
tensor([[1., 1., 1.],
        [1., 1., 1.]])



RuntimeError: The expanded size of the tensor (300) must match the existing size (3) at non-singleton dimension 1

In [20]:
batch

[SOURCE: kann ich nur weiter empfehlen .
 INSERT EXACT: [u'buch', u'das', u'jedem']
 DELETE EXACT: [u'weiter']
 TARGET: ich kann das buch nur jedem empfehlen .,
 SOURCE: kann ich echt nur empfehlen .
 INSERT EXACT: [u'<org>', u'buch', u'das']
 DELETE EXACT: [u'echt']
 TARGET: ich kann das buch nur empfehlen . <org>]

In [22]:
import torch
mask = torch.tensor([[1., 1., 1.], [1., 1., 1.]])

In [23]:
mask.shape

torch.Size([2, 3])

In [24]:
mask.expand(3, 300)

RuntimeError: The expanded size of the tensor (300) must match the existing size (3) at non-singleton dimension 1